In [102]:
from numpy import sqrt
import pandas as pd

from risk_lib.data import get_factor_data, calculate_returns_set, get_factor_master
from risk_lib.chart import draw_volatility, draw_correlation, draw_beta
from risk_lib.config import HALFLIFES


In [ ]:
# halflifes = [21, 63, 126, 252, 512]
data = get_factor_data()
# factor_master = pd.DataFrame(data['factor_name'].attrs).T
# factor_list = factor_master.index.to_list()
data

In [ ]:
factor_name = 'QQQ'
factor_name_1 = 'IWM'

In [ ]:
from risk_lib.chart import px_line
da = data.cret.sel(factor_name=['SPY', 'IWM'])
px_line(da, title='SPY vs IWM', x='date', y='cret', color='factor_name')

In [ ]:
halflifes = HALFLIFES

draw_correlation(data.corr, factor_name=factor_name, factor_name_1=factor_name_1, corr_type=halflifes).show()
draw_volatility(data.vol, factor_name=factor_name, vol_type=halflifes).show()
draw_volatility(data.vol, factor_name=factor_name_1, vol_type=halflifes).show()

# Calculate week-over-week returns

In [ ]:
cret = data.cret.to_pandas()
# cret.sort_index(ascending=False).to_clipboard()

In [ ]:
factor_master = get_factor_master()

In [ ]:
factor_master['multiplier']

In [ ]:
ret_1d = calculate_returns_set(cret, periods=1, 
                      diffusion_map=factor_master['diffusion_type'], 
                      multiplier_map=factor_master['multiplier'])

ret_5d = calculate_returns_set(cret, periods=5, 
                      diffusion_map=factor_master['diffusion_type'], 
                      multiplier_map=factor_master['multiplier'])


In [ ]:
from risk_lib.data import get_factor_master, get_factor_data
data = get_factor_data()
factor_master = get_factor_master() #data)
factor_master

In [ ]:
# from risk_lib.stats import get_volatility_set_new
# from risk_lib.config import HALFLIFES
# vol_1d = get_volatility_set_new(cret, halflifes=HALFLIFES)

# Betas

In [ ]:
COV_TYPES = {str(h): {'vol_type': h,
                      'corr_type': h}
             for h in HALFLIFES}
COV_TYPES

[(v['vol_type'], v['corr_type']) for k, v in COV_TYPES.items()]

In [ ]:
vol_0 = data.vol.sel(factor_name=factor_name).rename({'vol_type': 'cov_type'})
vol_1 = data.vol.sel(factor_name=factor_name_1).rename({'vol_type': 'cov_type'})
corr  = data.corr.sel(factor_name=factor_name, factor_name_1=factor_name_1).rename({'corr_type': 'cov_type'})
beta = (corr * vol_1 / vol_0).rename('beta')
beta

draw_beta(data, factor_name=factor_name, factor_name_1=factor_name_1).update_yaxes(type="log")

# Returns chart

In [ ]:
factor_data = get_factor_data()

In [ ]:
def draw_returns(ret, factor_1, factor_2):
    df = ret[[factor_1, factor_2]].reset_index()
    return px.bar(df, template='plotly_white', barmode='group')

In [ ]:
import plotly.express as px
factor_1 = 'QQQ'
factor_2 = 'IWM'

ret = factor_data.ret.to_pandas()[[factor_1, factor_2]]

df = ret.reset_index()
df
# px.bar(df, x='date', y=factor_1)
# px.bar(df, x='date', y=factor_2)

px.bar(ret[[factor_1, factor_2]].loc['2025':], template='plotly_white', barmode='group').update_traces(marker_line_width=0)
# px.bar(df, x='date', y=factor_1)

In [ ]:
df = factor_data.ret.to_pandas()[[factor_name, factor_name_1]] #.loc['2020':]
px.bar(df, template='plotly_white', barmode='group').show()

df2 = factor_data.ret.to_pandas()[[factor_name, factor_name_1]].reset_index() #.loc['2025':]
px.bar(df2, x='date', y=[factor_name, factor_name_1], template='plotly_white', barmode='group').update_traces(marker_line_width=0).show()


# Calcuate zscore

In [83]:
data.vol.sel(vol_type=21).to_pandas()

factor_name,SPY,IWM,MDY,RSP,QQQ,DIA,XLK,XLI,XLF,KRE,...,VXX,^VIX3M,^MOVE,TRUMP,FED,AI,GEOPOLITICS,2s10s,XLPXLY,TRADEWAR
date,,,,,,,,,,,,,,,,,,,,,
1993-01-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-02-02,5.606561,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-02-03,6.793416,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-02-04,5.846003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-22,39.078020,36.819353,38.125928,32.695762,44.430176,31.685459,49.935504,35.207364,35.258490,43.386776,...,130.046208,163.154282,86.473677,69.168940,4.092990,49.129984,21.723132,4.627993,34.717181,18.473138
2025-04-23,38.746139,36.541171,37.730369,32.310569,44.262211,31.353879,49.931475,34.849980,34.862394,43.236912,...,128.634243,161.409894,85.242783,69.648003,4.060606,49.215895,22.557063,4.747755,35.284189,19.310857
2025-04-24,38.627682,36.469739,37.667467,32.182633,44.328394,31.101176,50.340861,34.906236,34.442138,42.832537,...,127.229647,159.892141,86.553347,68.644318,4.096163,49.403678,22.387463,4.688580,35.799172,19.011909


In [86]:
data.vol.sel(vol_type=21).shift({'date': 1}).to_pandas()

factor_name,SPY,IWM,MDY,RSP,QQQ,DIA,XLK,XLI,XLF,KRE,...,VXX,^VIX3M,^MOVE,TRUMP,FED,AI,GEOPOLITICS,2s10s,XLPXLY,TRADEWAR
date,,,,,,,,,,,,,,,,,,,,,
1993-01-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-02-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-02-03,5.606561,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-02-04,6.793416,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-22,38.860570,36.477131,37.917433,32.327634,44.374861,31.063661,50.123339,35.356464,34.407785,42.925080,...,129.780851,163.825562,84.236176,68.938635,4.156608,49.208066,21.855172,4.704501,34.751589,18.019027
2025-04-23,39.078020,36.819353,38.125928,32.695762,44.430176,31.685459,49.935504,35.207364,35.258490,43.386776,...,130.046208,163.154282,86.473677,69.168940,4.092990,49.129984,21.723132,4.627993,34.717181,18.473138
2025-04-24,38.746139,36.541171,37.730369,32.310569,44.262211,31.353879,49.931475,34.849980,34.862394,43.236912,...,128.634243,161.409894,85.242783,69.648003,4.060606,49.215895,22.557063,4.747755,35.284189,19.310857


In [108]:
data.ret.to_pandas().div(100)

factor_name,SPY,IWM,MDY,RSP,QQQ,DIA,XLK,XLI,XLF,KRE,...,VXX,^VIX3M,^MOVE,TRUMP,FED,AI,GEOPOLITICS,2s10s,XLPXLY,TRADEWAR
date,,,,,,,,,,,,,,,,,,,,,
1993-01-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-02-01,0.711286,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-02-02,0.211814,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-02-03,1.057007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-02-04,0.418517,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-22,2.601774,2.588375,2.512794,2.465655,2.625205,2.685918,2.437727,1.747081,3.307773,3.212450,...,7.292204,7.477220,7.941816,4.022652,-0.014856,2.657420,-0.791431,-0.065844,-1.746136,-1.475745
2025-04-23,1.549546,1.482903,1.277335,0.967418,2.267815,1.030956,2.899297,1.251709,1.187450,2.253199,...,3.608691,4.855689,1.478665,4.620071,-0.136490,3.069284,-2.274107,0.386327,-2.806138,-2.004103
2025-04-24,2.104893,2.002627,2.133473,1.691940,2.813712,1.280596,3.731569,2.242651,1.110643,1.680675,...,3.685283,5.638829,7.135506,1.064751,0.364149,3.366067,1.242344,0.073281,-2.871890,-0.059232


In [104]:
data.vol.sel(vol_type=63).to_pandas() / sqrt(252)

factor_name,SPY,IWM,MDY,RSP,QQQ,DIA,XLK,XLI,XLF,KRE,...,VXX,^VIX3M,^MOVE,TRUMP,FED,AI,GEOPOLITICS,2s10s,XLPXLY,TRADEWAR
date,,,,,,,,,,,,,,,,,,,,,
1993-01-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-02-02,0.353180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-02-03,0.425926,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-02-04,0.367662,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-22,1.701128,1.780361,1.729761,1.436840,2.014585,1.404477,2.309113,1.587263,1.617805,2.262485,...,6.465235,8.010062,4.806706,3.752567,0.247637,2.254717,1.114878,0.253911,1.739254,0.864519
2025-04-23,1.700052,1.778385,1.726121,1.432868,2.018321,1.401349,2.317793,1.584264,1.613545,2.262984,...,6.444440,7.987272,4.783804,3.765513,0.246992,2.265923,1.137543,0.256957,1.756469,0.888469
2025-04-24,1.705427,1.782097,1.731958,1.436358,2.029272,1.400475,2.339056,1.593190,1.608639,2.257678,...,6.424025,7.970517,4.819504,3.746942,0.247933,2.281035,1.137167,0.255952,1.773939,0.883791


In [ ]:
zscore = data.ret / data.vol.shift({'date': 1})
zscore
# zscore.to_pandas()

<xarray.DataArray (date: 8117, factor_name: 68, vol_type: 5)> Size: 22MB
dask.array<truediv, shape=(8117, 68, 5), dtype=float64, chunksize=(2030, 17, 2), chunktype=numpy.ndarray>
Coordinates:
  * factor_name  (factor_name) object 544B 'SPY' 'IWM' ... 'XLPXLY' 'TRADEWAR'
  * date         (date) datetime64[ns] 65kB 1993-01-29 1993-02-01 ... 2025-04-28
  * vol_type     (vol_type) int64 40B 21 63 126 252 512